<img src="./Figures/LogoObsera.png" width="75" align="left" style="vertical-align:middle;margin:0px 100px" ><h1>   River load sensor calibration</h1>


<h2>Find the regression for linking suspended load with high frequency turbidity</h2>

-------

- **Authors**: Antoine Lucas (lucas@ipgp.fr) - Amande Roque-Bernard (roquebernard@ipgp.fr) - Grégory Sainton (sainton@ipgp.fr)
- **Version**: 2.0


--------
- **Objective**: We seek to calibrate a turbidity probe placed in a river against independent measurements of suspended solids concentration.

- **The data**: The data come from ObsEra, an observatory located in Guadeloupe that gives us information on the erosion of this volcanic island in a few watersheds.

- **You will learn**: how to prepare and clean the data. It's probably the most important part of the job. Then, you will play with your first models with the `scikit-learn` library. 


- **NB1**: If some errors or bugs are still in the notebook despite our efforts, please send us an email to tell us.

- **NB2**: Solutions which will be proposed in the corrected notebooks are only a possible solution. 

- **NB3**: This notebook is based on Amande's thesis work. Further analysis can be done, please contact her. 

----
## This notebook is made of two parts 
### How to open the data ? 

It's a very guided part to learn how to have a first look on your datasets. School data are often already clean but the real data are usually very messy. So we need to spend time to clean and to prepare the data before starting the real analysis.

### Select, apply, train a model

In this section, we will play with some models using `scikit-learn` library. 
1. Linear regression
2. Principal Component Analysis (PCA)

But of course, we encourage you to go further, to test some other models and to compare your results.

Have fun !

----


## How to open the data

All the data are saved in the `data` directory of this lab.
They are separeted in two subsets, one with chemical informations and another one with hydrological informations

This part is done for you. No need to touch the following cell. Just notice that `filelist_chem` and `filelist_hydro` are the 2 array variables which contain the list of files we will use after.

In [ ]:
### DON'T MODIFY ANYTHING BELOW
#%reset -f     
# The previous line is used to reset all the variables at each runs

import os, sys
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

#Chemical data
ObseraDir_chem = './data/CHEM/'
filelist_chem = glob(ObseraDir_chem + 'C*.csv')

#Hydrologic data
ObseraDir_hydro = './data/HYDRO/'
filelist_hydro = glob(ObseraDir_hydro + 'Data*.csv' )  # list of the raw data files        

### Observe and prepare the data
The first step, is to look at your data. 

- For the first dataset, you'are guided at almost every steps. 
- For the second data set, you will have to do it on your own.

#### Chemical data

##### Read the data

1. Using the `read_csv` function from Pandas library, open the file (Use the parameter `na_value` to remove NaN) data.

2. Create a "datetime" column by adding together the columns "Date" and "Hour"   ```2020-01-06 10:00:00```
3. Convert this coloumn to a datatime format.
4. Attribute the column `datetime` as the index of your pandas structure.

**hints** 
- Beware of the separator which is a semicolon (;)
- `filelist_chem` is an array

In [ ]:
# WRITE YOUR CODE HERE
data_chem = None                           # open the file 

##### Inspect the data
Using `data_chem.head()`, `data_chem.info()`,`data_chem.describe()`, explore your data set


- numbers of rows, of columns
- checks the range of each fields
- What are the different fields, their type (especially, non numeric values).
- Do we have NaN, blank values ?

In [ ]:
# WRITE YOUR CODE HERE

##### Extract datetime and suspended Load data only

In fact, for this exercice, we only need to keep the fields `datetime` and `Suspended Load` and `Level`

1. Create a new pandas DataFrame only containing
2. Remove negative value and NaN value
3. Plot the `Suspended Load` in function of the `datetime` (You can use either Matplotlib or Seaborn (or whatever))

**hints**
- One can remove negative value by creating a mask
- For the NaN removal, use the Pandas `.dropna()` function

In [ ]:
# WRITE YOUR CODE HERE

Check that your plot looks like 
<img src="./Figures/Plot_SuspendedLoadOvertime.png" width = "300">

#### Hydrological data

In this second part, you will have to deal with the files in the variable `filelist_hydro`

1. Open all the files and create a unique DataFrace called `data_hydro`.
2. Inspect your data.
3. Plot the `turbidity` in function of the `datetime`


**hints**
- Create a first DataFrame with the first file and then concatenate some other DataFrame made with the other files
- Think about using `pd.concat` to concatenate the DataFrame

In [ ]:
# WRITE YOUR CODE HERE

## Synchronize the data
The goal here is to get the same time vector on the both structure. But the sampling is not the same for the both set of data.

To ease your life, the next part is done for you. Keep the method somewhere in case you need to use it someday.

- We first check the common interval
- Upsample the date to get the same sampling as data_chem
- Match the data 

### Synchronisation : the tricks 

You just have to execute the following cell.

In [ ]:
mask = data_chem['datetime'] >= data_hydro.datetime[0] # premiere date de l'hydro
data_chem = data_chem.loc[mask]

del mask

mask    = data_chem['Suspended Load'] >= 0.
data_ch = data_chem.loc[mask]

turbidity   = data_hydro[["turbidity"]]
water_level = data_hydro[["level"]]

upsampled   = turbidity.resample('1T')
turbidity_by_minute = upsampled.interpolate(method='linear')

upsampled   = water_level.resample('1T')
water_level_by_minute = upsampled.interpolate(method='linear')

del upsampled

# How to use pandas to find consecutive same data in time series
# https://stackoverflow.com/questions/26911851/how-to-use-pandas-to-find-consecutive-same-data-in-time-series

turbidity['value_grp'] = (np.isnan(turbidity.turbidity)).astype('int')
turbidity['value_grp'] = (turbidity['value_grp'].diff(1) != 0).astype('int').cumsum()
turbidity['Date'] = turbidity.index


water_level['value_grp'] = (np.isnan(water_level.level)).astype('int')
water_level['value_grp'] = (water_level['value_grp'].diff(1) != 0).astype('int').cumsum()
water_level['Date']      = water_level.index



# decoupage par paquets de donnees de turbidity
check = pd.DataFrame({'BeginDate' : turbidity.groupby('value_grp').Date.first(), 
              'EndDate' : turbidity.groupby('value_grp').Date.last(),
              'Consecutive' : turbidity.groupby('value_grp').size(),
              'sum' : turbidity.groupby('value_grp').turbidity.sum().astype('float')}).reset_index(drop=True)   

    
for i in range(len(check)):
    if check['sum'][i] > 0. : # le morceau contient des donnees car la somme des valeurs est positive
        continue
    else:
        if check['Consecutive'][i] > 3. :  # le chunk contient trop de nan pour une interpolation correct 
                                           # donc valeurs de linterpolation changé en NAN
            turbidity_by_minute[check['BeginDate'][i]:check['EndDate'][i]] = np.NaN
            water_level_by_minute[check['BeginDate'][i]:check['EndDate'][i]] = np.NaN

       
common = turbidity_by_minute.index.intersection(data_ch['Suspended Load'].index)
suspended_load = data_ch['Suspended Load'].loc[common]

turbidity_by_minute            = turbidity_by_minute.loc[common]
matching_turbidity_by_minute   = turbidity_by_minute.loc[common]
matching_water_level_by_minute = water_level_by_minute.loc[common]

turbidity_by_minute   = matching_turbidity_by_minute        
water_level_by_minute = matching_water_level_by_minute    

In [ ]:
fig = plt.figure(figsize=(8,6))

ax1 = plt.subplot(311)
plt.plot(suspended_load[:],'.')
plt.grid()
plt.ylabel('$C_{S}$ [mg/L]')

ax2 = plt.subplot(312, sharex=ax1)
plt.plot(matching_turbidity_by_minute,'.')
plt.ylabel('T [NTU]')
plt.grid()
fig.tight_layout()

ax3 = plt.subplot(313, sharex=ax1)
plt.plot(matching_water_level_by_minute,'.')
plt.ylabel('h [cm]')
plt.grid()
fig.tight_layout()

In [ ]:
SS   = np.array(suspended_load)
TUR  = np.array(matching_turbidity_by_minute)
H    = np.array(matching_water_level_by_minute)

SS  = np.squeeze(SS)
TUR = np.squeeze(TUR)
H   = np.squeeze(H)

mask = ~np.isnan(TUR)

SS   = SS[mask]
TUR  = TUR[mask]
H    = H[mask]

TUR1,SS = zip(*sorted(zip(TUR,SS)))
TUR,H = zip(*sorted(zip(TUR,H)))

del TUR1

In [ ]:
data = pd.DataFrame({'turbidity':TUR, 'level':H, 'Suspended Load': SS})

### Questions

The final DataFrame you will have to deal with is `data`

**Have a quick view on your data**

Using `scatter_matrix` from pandas, one can have a look on all the numerical variables of the dataset. 
- Diagonals show the distribution of the variable
- Other combinations show the scatter plot of a given variable vs another one.


**Make two plots (horizontal subplots)**:
1. Suspended Load (Units = $C_s(mg/L)$) in function of the Turbidity (Units = $NTU$)
2. Suspended Load in function of the Level (Units = cm)


In [ ]:
# Have a quick view on your data -> WRITE YOUR CODE HERE

In [ ]:
# Make two plots -> WRITE YOUR CODE HERE

Check that your plot looks like


<img src="./Figures/Plot Cs_h.png">

## Select, apply, train a model

It's now time to fit the model. We are trying several models...


But before, let us give you a **Quick remind about the Scikit-Learn Design** 
It's a quote from Aurelien Geron, _Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow_


** Quick remind about the Scikit-Learn Design** (from A. Geron, _Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow_)

Scikit-Learn’s API is remarkably well designed. The main principles are 

**Consistency**. All objects share a consistent and simple interface:

- **Estimators**. Any object that can estimate some parameters based on a dataset is called an estimator (e.g., an imputer is an estimator). The estimation itself is performed by the fit() method, and it takes only a dataset as a parameter (or two for supervised learning algorithms; the second dataset contains the labels). Any other parameter needed to guide the estimation process is con‐ sidered a hyperparameter (such as an imputer’s strategy), and it must be set as an instance variable (generally via a constructor parameter).

- **Transformers**. Some estimators (such as an imputer) can also transform a dataset; these are called transformers. Once again, the API is quite simple: the transformation is performed by the transform() method with the dataset to transform as a parameter. It returns the transformed dataset. This transforma‐ tion generally relies on the learned parameters, as is the case for an imputer. All transformers also have a convenience method called fit_transform() that is equivalent to calling fit() and then transform() (but sometimes fit_transform() is optimized and runs much faster).

- **Predictors**. Finally, some estimators are capable of making predictions given a dataset; they are called predictors. For example, the LinearRegression model in the previous chapter was a predictor: it predicted life satisfaction given a country’s GDP per capita. A predictor has a predict() method that takes a dataset of new instances and returns a dataset of corresponding predictions. It also has a score() method that measures the quality of the predictions given a test set (and the corresponding labels in the case of supervised learning algorithms).

- **Inspection**. All the estimator’s hyperparameters are accessible directly via public instance variables (e.g., imputer.strategy), and all the estimator’s learned parameters are also accessible via public instance variables with an underscore suffix (e.g., imputer.statistics_).

- **Nonproliferation of classes**. Datasets are represented as NumPy arrays or SciPy sparse matrices, instead of homemade classes. Hyperparameters are just regular Python strings or numbers.

- **Composition**. Existing building blocks are reused as much as possible. For example, it is easy to create a Pipeline estimator from an arbitrary sequence of transformers followed by a final estimator, as we will see.

- **Sensible defaults**. Scikit-Learn provides reasonable default values for most parameters, making it easy to create a baseline working system quickly.


### Linear regression

No need to use **scikit-learn** to do linear regression but let's learn to do with this library anyway as an introduction to it.

To import the Linear Regression methods : 
`from sklearn.linear_model import LinearRegression`

#### Questions

1. Apply a classic linear regression on the data : $SS = f(TUR)$ 

    - Find the coefficients of the fit
    - Give the score of the result
    

2. How good is your model ? 

    - Apply your model to predict values using your input dataset: `myprediction = model.predict(dataset)`
    - Measure the **MAE** (Mean Absolute Error) to evaluate the quality of your model: 
`from sklearn.metrics import mean_absolute_error`

`mymae = mean_absolute_error(myprediction-labeldata)`

In our case, `labeldata` is the Suspended Load vector


3. Check your result with the log or your data to see if the results are improved.



In [ ]:
# WRITE YOUR CODE HERE

#Remane the vectors

TUR= data["turbidity"].to_numpy().reshape(len(data["turbidity"]),1)
SS = data["Suspended Load"].to_numpy().reshape(len(data["Suspended Load"]),1)


Now, try to fit your data in the other direction

In [ ]:
# WRITE YOUR CODE HERE

### Robust linear regression

As you can see, the data have some outiers. Is is possible to improve the result of the fit ? 

Just as an example, apply a RANSAC regression on the data.
- Coefficient ? 
- Plot of the fit
- Score of the result
- Comparison with the classical LR -> Comments ?

In [ ]:
# WRITE YOUR CODE HERE

##### Plot, comparison, conclusion

1. Plot the data and your different fits on the same plot
2. Any comments about the linear regression on these data ?

### Introduction to PCA

As you've seen during the lectures, *Principal Component Analysis* (PCA) is by far the most popular dimensionality reduction algorithm. First it identifies the hyperplane that lies closest to the data, and then it projects the data onto it.

But what lower dimensional hyperplane to choose ? We have to choose one axis which preserve the maximum variance. From math point of view, it is the axis which minimizes the mean square distribution between the original data and its projection on the axis. 


In this part, we are going to apply the PCA to modelise the relation between the turbidity and the suspended load. 


#### Quick remind on what the PCA is. 

By definition, the PCA is the way to project the data in a lower dimension base. The base is choosed in order to maximize the variance of the projected data.

For example: 

- Let's say that we have data in 2 dimensions ($D=2$) and we want to find project into a single dimension space ($M=1$ with the vector $u_1$). 

The variance of the projected data in $M=1$ is 

$\begin{equation}
V = \frac{1}{N}\sum\limits_{n=1}^{N} (u_1^Tx_n -u_1^T\bar{x})^2 \tag{1}
\end{equation}$ 

where $u_1^Tx_n$ is the projection of the point $x_n$ in the $u_1$ space. 

$\begin{align*}
V &= \frac{1}{N}\sum\limits_{n=1}^{N} (u_1^T(x_n -\bar{x}))^2 \\
          &=\frac{1}{N}\sum\limits_{n=1}^{N} u_1^T.(x_n -\bar{x}).(x_n -\bar{x})^T.u_1 \\
          &=u_1^T.\big\lbrace\frac{1}{N}\sum\limits_{n=1}^{N} (x_n -\bar{x}).(x_n -\bar{x})^T)\big\rbrace.u_1 \\ \\
          &= u_1^T.S.u_1
\end{align*} \tag{2}$ 

where $S$ is the covariance matrix and $\bar{x}$ is the average of the data $x_n$. 

Now, we have to maximize the variance. We constraint the norm of $u_1$ to be equal to $1$ and then, we maximize.

It can be easely expressed with the [Lagrangian multipliers](https://en.wikipedia.org/wiki/Lagrange_multiplier). 

$\begin{equation}
\mathcal{L} = u_1^T.S.u_1 + \lambda_1(1-u_1^Tu_1)
\tag{3}
\end{equation} $ 

Then we estimate the gradient in regards to $u_1$ which is equal to $0$.

Finally after calculations, we obtain:

$\begin{equation}
Su_1 = \lambda_1u_1
\tag{4}
\end{equation}$ 

It means that $u_1$ is the eigenvector of $S$ and $\lambda_1$ the eigenvalue. 

So, if $u_1$ is an eigenvector of $S$, the variance is equal to:

$\begin{equation}
V= u_1^T.S.u_1 = u_1^T.(\lambda_1.u_1) = \lambda_1
\tag{5}
\end{equation}$ 

So, to maximize the variance, we need to take de eigenvector $u_1$ corresponding to the highest eigenvalue $\lambda_1$.

- In case $M>1$, we are dealing the problem iterativaly. Each time, we are looking for another projection which maximize the variance and which is orthogonal to the previous ones. 

The projected data $y(x)$ on the $M$ dimensionnal base is finally given by the following expression:

$\begin{equation}
y(x) = (U_{:,1:M})^T.x
\tag{6}
\end{equation}$ 
where $U$ is the matrix of the eigenvectors of the covariance matrix $S$.


#### Back to the problem... 

Let's reduce the notation by using the two following variables for the turbidity and the suspended load values.

In [ ]:
TUR = data["turbidity"]
SS  = data["Suspended Load"]

#### Questions
1. Using `numpy`, estimate the covariance matrix of the log of theses two vectors
2. Find the Eigen values and the Eigen vectors of the covariance matrix.
3. Calculate the coefficients of the linear equation using the Eigen values.

**hint**
`np.cov()` and `np.log` and `np.linalg.eig` could be useful here

In [ ]:
# WRITE YOUR CODE HERE

In fact the relation between the turbidity and the suspended load can be modelised as a _power low_


SL_acp = np.exp(b_acp)*(TUR_x**a_acp)

#### Questions
1. Create a turbidity vector `TUR_x` from $[0,1000]$
2. Create a vector `SL_acp` using the coefficients of the PCA over the values of `TUR_x`

In [ ]:
# WRITE YOUR CODE HERE

Check that your plot looks like

<img src="./Figures/Plot_PCA_fit.png" width="500">

### PCA with Scikit-learn (bonus)

This section is just to show you that everything you have calculated using the linear algebra is already implemented in `scikit-learn`

The next cells are just here to show you how to do it. Of course, don't hesitate to refer to the documentation about PCA = 
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html